In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# https://www.census.gov/topics/education/educational-attainment/data/tables.2010.List_2016040495.html

# Testing

## Data Model

In [ ]:
twenty_one_df = pd.read_csv(Path('../race/educational/2021/model_data/2021_model_data.csv'))


In [14]:
X_train = pd.get_dummies(twenty_one_df.drop(columns=['Sex']))
y_train = twenty_one_df['Sex']

In [15]:
X_train

,Total,None,1st - 4th grade,5th - 6th grade,7th - 8th grade,9th grade,10th grade,11th grade2,High school graduate,"Some college, no degree",...,Age_ 50 to 54 years,Age_ 55 to 59 years,Age_ 60 to 64 years,Age_ 65 to 69 years,Age_ 70 to 74 years,Age_ 75 years and over,Race_Asian,Race_Black,Race_Hispanic,Race_White
0,881,4,0,0,3,6,12,91,117,355,...,0,0,0,0,0,0,1,0,0,0
1,766,0,0,0,3,0,3,10,73,63,...,0,0,0,0,0,0,1,0,0,0
2,874,2,0,7,7,2,0,2,80,51,...,0,0,0,0,0,0,1,0,0,0
3,910,0,0,2,8,3,4,6,125,71,...,0,0,0,0,0,0,1,0,0,0
4,805,3,2,2,19,1,5,5,109,78,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,7945,18,84,111,119,99,148,234,2459,1182,...,0,1,0,0,0,0,0,0,0,1
92,8258,33,91,105,95,112,108,235,2647,1277,...,0,0,1,0,0,0,0,0,0,1
93,7126,26,57,96,86,52,88,170,1970,1218,...,0,0,0,1,0,0,0,0,0,1
94,5771,33,42,62,85,85,90,104,1421,978,...,0,0,0,0,1,0,0,0,0,1


In [16]:
y_train

0     Female
1     Female
2     Female
3     Female
4     Female
       ...  
91      Male
92      Male
93      Male
94      Male
95      Male
Name: Sex, Length: 96, dtype: object

In [17]:
y_train.value_counts()

Female    48
Male      48
Name: Sex, dtype: int64